In [ ]:
try:
    import cv2
    import pytesseract
    import numpy as np
except ImportError:
    print("""
        Make sure you have the following packages installed: opencv-python, pytesseract and numpy.\n
        You can do this by typing the following in your python environment:\n
        pip install opencv-python pytesseract numpy
    """)

import os
import csv
import time
import re
from datetime import date

In [ ]:
#Images directory and csv files result path
images_folder = "images/"
csv_files_path = "csv_results/" # YOU CAN CHANGE IT
csv_name = str(date.today()) + '.csv'

#Creates the csv results folder if not exists
if not os.path.exists(csv_files_path):
    os.makedirs(csv_files_path)

# Check if the new data already exists in the csv file
def data_already_exists(file_path, new_data):
    if not os.path.isfile(file_path):
        return False
    
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Checando se todos os valores relevantes já estão no arquivo
            if (row['filename'] == new_data['filename'] and
                float(row['angle1']) == new_data['angle1'] and
                float(row['angle2']) == new_data['angle2'] and
                float(row['total']) == new_data['total']):
                return True
    return False

# Image processing
for imagefile in os.listdir(images_folder):
    img = cv2.imread(images_folder + imagefile)
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    thresh = cv2.threshold(img_gray, 230, 255, cv2.THRESH_BINARY)[1]
    
    thresh2 = thresh.copy()
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    pieces = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        pieces.append(img[y:y+h, x:x+w])
    
    pieces_gray = [cv2.cvtColor(piece, cv2.COLOR_BGR2GRAY) for piece in pieces]
    
    texts = []
    custom_config = r'--oem 3 --psm 11 -c tessedit_char_whitelist=0123456789.'
    results = []
    for piece in pieces_gray:
        numbers_string = pytesseract.image_to_string(piece, config=custom_config)
        numbers = re.sub(r'[a-z\n]', '', numbers_string.lower())
        try:
            n = float(numbers)
            results.append(n)
        except:
            pass
        
    print(f"Found angles for {imagefile}: {results}")

    data = {
        "timestamp": time.time(),
        "filename": imagefile,
        "angle1": results[0],
        "angle2": results[1],
        "total": round(sum(results),2)
    }

    # Check if the new data already exists in the csv file
    if not data_already_exists(csv_files_path + csv_name, data):
        print(f"Inserting new data: {data}")
        
        # Open csv file for writing
        with open(csv_files_path + csv_name, 'a+', newline='') as csvfile:
            fieldnames = ['timestamp', 'filename','angle1', 'angle2', 'total']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Write the header if the file didn't exist
            if os.stat(csv_files_path + csv_name).st_size == 0:  # Verifica se o arquivo está vazio
                writer.writeheader()
                
            # Write the new data into the csv file
            writer.writerow(data)
    else:
        print(f"Data already exists, ignoring: {data}")
